In [3]:
import sys
import numpy as np
import mxnet as mx
import tensorflow as tf
from datascience import *

In [58]:
lyrics = Table().read_table('lyrics.csv')
altered = lyrics.select(np.arange(2, lyrics.num_columns))
altered.to_csv('genreAndProp.csv')
# propsOnly = lyrics.select(np.arange(3, lyrics.num_columns))
# genresOnly = Table().with_columns('Country', lyrics.apply(lambda x: ((int) (x=='Country')) , 'Genre'), 'Hip-hop', lyrics.apply(lambda x: ((int) (x=='Hip-hop')) , 'Genre'))
# propsOnly.to_csv('lyricsProps.csv')
# genresOnly.to_csv('lyricsGenres.csv')

Our input vectors are of length 4816, because we have 4816 words.
Our output vectors are of length 2, and are a softmax. 1 in the first position means country, 1 in the second position means hip hop.

In [57]:
filename_queue = tf.train.string_input_producer(["lyricsProps.csv", "lyricsGenres.csv"], num_epochs = 6)

reader = tf.TextLineReader(skip_header_lines=1)
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = [[0] for _ in range(4819)]
reading = tf.decode_csv(value, record_defaults=record_defaults)
features = tf.stack(reading[0:len(reading)-2])
genres = tf.stack(reading[len(reading)-2:])

In [46]:
def create_file_reader_ops(filename_queue):
    reader = tf.TextLineReader(skip_header_lines=1)
    _, csv_row = reader.read(filename_queue)
    record_defaults = [[0] for _ in range(4819)]
    reading = tf.decode_csv(csv_row, record_defaults=record_defaults)
    features = tf.stack(reading[0:len(reading)-2])
    genres = tf.stack(reading[len(reading)-2:])
    return features, genres

In [ ]:
directory = "genreAndProp.csv"
filename_queue = tf.train.string_input_producer(
    tf.train.match_filenames_once(directory),
    shuffle=True)

# Each file will have a header, we skip it and give defaults and type information
# for each column below.
line_reader = tf.TextLineReader(skip_header_lines=1)

_, csv_row = line_reader.read(filename_queue)

# Type information and column names based on the decoded CSV.
record_defaults = [[""]] + [[0.0] for _ in range(4817)]
readings = tf.decode_csv(csv_row, record_defaults=record_defaults)
# Turn the features back into a tensor.
features = tf.stack(readings[1:])
genre = readings[0]

with tf.Session() as sess:
    tf.global_variables_initializer.run()

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    # We do 10 iterations (steps) where we grab an example from the CSV file. 
    for iteration in range(1, 11):
        # Our graph isn't evaluated until we use run unless we're in an interactive session.
        example, label = sess.run([features, genre])

        print(example, label)
    coord.request_stop()
    coord.join(threads)

In [64]:
all_genres = ["Country", "Hip-hop"]
onehot = {}
# Target number of species types (target classes) is 3 ^
genre_count = len(all_genres)

# Print out each one-hot encoded string for 3 species.
for i, genre in enumerate(all_genres):
    # %0*d gives us the second parameter's number of spaces as padding.
    print("%s,%0*d" % (genre, genre_count, 10 ** i))

Country,01
Hip-hop,10


In [54]:
x = tf.placeholder(tf.float32, [None, 4817])
W = tf.Variable(tf.zeros([4817, 2]))
b = tf.Variable(tf.zeros([2]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [55]:
y_ = tf.placeholder(tf.float32, [None, 2])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [66]:


# sess = tf.InteractiveSession()
# tf.global_variables_initializer().run()
# for _ in range(1000):
#   batch_xs, batch_ys = mnist.train.next_batch(100)
#   sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    while True:
        try:
            example_data, genre_data = sess.run(train_step, feed_dict={x: features, y_: genre})
            print(example_data, genre_data)
        except tf.errors.OutOfRangeError:
            break
    

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled


Applications                Pictures
Desktop                     Public
Documents                   Untitled.ipynb
Downloads                   java_error_in_idea_6093.log
InfinityFinancial           learning-git
Library                     lyrics.csv
LinkedList                  temp
Movies                      vectorLyrics.csv
Music
